<a href="https://colab.research.google.com/github/hamzamusht550/Amazon-Clone-Frontend/blob/main/MKV_Combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#@title Install Dependencies

from IPython.display import clear_output
from google.colab import drive
from google.colab import auth

!apt-get install -y mediainfo ffmpeg mkvtoolnix aria2
!pip install patool pymediainfo ffmpeg-python pyrogram tgcrypto nest_asyncio

drive.mount('/content/drive')
auth.authenticate_user()
clear_output()

In [ ]:

#@title Mediainfo MKV

MKV_INFO = "/content/drive/MyDrive/Solo Leveling (2024) S01E01 1080p BluRay 10bit HEVC [Hindi CR AAC 2.0 + English AAC 2.0 + Japanese AAC 2.0] x265 (DarQ-KiNGSMAN)_merged.mkv" #@param {type: "string"}

from IPython.display import clear_output
clear_output()

!ffprobe -i "{MKV_INFO}"
!mediainfo "{MKV_INFO}"
!du -h "{MKV_INFO}"
!du -BM "{MKV_INFO}"

In [4]:

#@title MKV Merger

MKV_FILES = "https://brucewayne.tgmkvofcl.workers.dev/225526?hash=AgADGg, https://brucewayne.tgmkvofcl.workers.dev/225528?hash=AgADGw, https://brucewayne.tgmkvofcl.workers.dev/225530?hash=AgADFw, https://brucewayne.tgmkvofcl.workers.dev/225532?hash=AgADHA, https://brucewayne.tgmkvofcl.workers.dev/225534?hash=AgADHQ, https://brucewayne.tgmkvofcl.workers.dev/225538?hash=AgADHw, https://brucewayne.tgmkvofcl.workers.dev/225540?hash=AgADGA, https://brucewayne.tgmkvofcl.workers.dev/225542?hash=AgADIQ, https://brucewayne.tgmkvofcl.workers.dev/225536?hash=AgADGQ, https://brucewayne.tgmkvofcl.workers.dev/225537?hash=AgADHg, https://brucewayne.tgmkvofcl.workers.dev/225549?hash=AgADJQ, https://brucewayne.tgmkvofcl.workers.dev/225545?hash=AgADIg, https://brucewayne.tgmkvofcl.workers.dev/225547?hash=AgADJA, https://brucewayne.tgmkvofcl.workers.dev/225548?hash=AgADJg, https://brucewayne.tgmkvofcl.workers.dev/225553?hash=AgADJw, "  # @param {type: "string"}
OUTPUT_DIR = ""  #@param {type: "string"}
MAX_FileSize = 2  #@param {type: "number"}
JUST_DOWNLOAD= False  #@param {type: "boolean"}
Delete_Extracted_DIR= False  #@param {type: "boolean"}
# Delete_Extracted_Data = False  #@param {type: "boolean"}
# #@markdown ***Start 00:00:00 / End 00:00:10***
# trim_start = ""  #@param {type: "string"}
# trim_end = ""  #@param {type: "string"}

from IPython.display import clear_output
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
import ffmpeg
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
drive.mount('/content/drive')
clear_output()


def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")

def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    listStr = '\n'.join(listStr.split()) # Remove newLines from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n\n")
    return finalList

def extract_file_info(file_path):
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def getMKVFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort(key=str.lower)
    return all_files


def extractArchive(Archive_File, extracted_folder=''):
    if not extracted_folder:
        extracted_folder = os.path.splitext(Archive_File)[0]
    if os.path.exists(extracted_folder):
        print(f"Archive File Already Exists:\n{extracted_folder}")
        return extracted_folder

    os.makedirs(extracted_folder, exist_ok=True)
    print(f"\nExtracting archive to: {extracted_folder}")
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    print(f"Extraction complete: {extracted_folder}\n")
    return extracted_folder


def runCommand(cmd, showLog=0):
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        print(f"Running command: {cmd}")
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()
    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return process


def filter_Filename(filename):
    MAX_FILENAME_LENGTH = 245
    # Replace unsuitable characters with a hyphen
    invalid_chars = r'[<>:"/\\|?*\']'
    valid_filename = re.sub(invalid_chars, '-', filename)
    # Truncate the filename if it exceeds the maximum length
    if len(valid_filename) > MAX_FILENAME_LENGTH:
        # Preserve the file extension if present
        if '.' in valid_filename:
            name, extension = valid_filename.rsplit('.', 1)
            truncated_name = name[:MAX_FILENAME_LENGTH - len(extension) - 1]  # Reserve space for the dot
            valid_filename = f"{truncated_name}.{extension}"
        else:
            valid_filename = valid_filename[:MAX_FILENAME_LENGTH]

    return valid_filename


def getResponseFileName(url):
    response = requests.get(url, stream=True)

    # Extract the filename from the Content-Disposition header
    filename = response.headers.get('content-disposition', '')
    if 'filename=' in filename:
        filename = filename.split("filename=")[-1].strip('"')
        return filter_Filename(filename)
    return ""


def Aria2Downloader(url, output_dir=''):
    """Download a file using Aria2."""
    print(f"Downloading file: {url}")
    Aria2 = f"aria2c --continue=true --file-allocation=none -x 16 -s 16 -d \"{output_dir}\" \"{url}\""
    process = subprocess.run(Aria2, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if process.returncode == 0:
        output_file_path = process.stdout.decode('utf-8')
        print(output_file_path)
        output_file_path = output_file_path.split("Download complete: ")
        output_file_path = output_file_path[1].split("\n")[0]
        print(f"\n\033[92mOutput ==> {output_file_path}\033[0m")  # Green color
        return output_file_path
    else:
        print(f"Download Error Status code({process.returncode}): {url}")

    return url


def mergeMKVs(video_paths, output_dir, max_size_gb):
    video_paths.sort(key=str.lower)
    if len(video_paths) < 2:
        print(video_paths, f" ==> Can't Merge {len(video_paths)} video")
        return video_paths

    os.makedirs(output_dir, exist_ok=True)

    base_name = extract_file_info(video_paths[0])[2] + "_merged.mkv"
    base_output_path = os.path.join(output_dir, base_name)

    max_size_bytes = max_size_gb * 1024**3

    # Create list.txt in the output directory
    list_path = os.path.join(output_dir, "list.txt")

    with open(list_path, "w") as f:
        for video_path in video_paths:
            abs_video_path = os.path.abspath(video_path)
            f.write(f"file '{abs_video_path}'\n")

    # Build FFmpeg command
    command = [
        "ffmpeg",
        "-f", "concat",
        "-safe", "0",
        "-i", list_path,
        "-c", "copy",
        "-f", "segment",
        "-segment_size", str(max_size_bytes),
        f"{base_output_path}%03d"
    ]

    command = f"ffmpeg -f concat -safe 0 -i \"{list_path}\" -map 0 -c copy \"{base_output_path}\""

    # Run command
    runCommand(command, 1)


def main(filesListStr, output_dir, max_size_gb):
    Files_List = strToList(filesListStr, ",")
    _File = Files_List[0]

    if len(Files_List) == 0 or (len(Files_List) == 1 and os.path.exists(_File) and _File.endswith('.mkv')):
        print(Files_List, " ==> Give At Least 2 files for Merging")
        exit()

    if len(Files_List) == 1:
        _File = Files_List[0]
        if _File.startswith('http'):
            _File = Aria2Downloader(_File)
        if os.path.exists(_File):
            if patoolib.is_archive(_File):
                _Folder = extractArchive(_File)
                Files_List = getMKVFilesList(_Folder)
            elif os.path.isdir(_File):
                Files_List = getMKVFilesList(_File)

    valid_Files_List = []
    if len(Files_List) > 1:
        for i, valid_File in enumerate(Files_List):
            if valid_File.startswith('http'):
                valid_File = Aria2Downloader(valid_File)
            if os.path.exists(valid_File):
                if valid_File.endswith('.mkv'):
                    valid_File = os.path.abspath(valid_File)
                    valid_Files_List.append(valid_File)
                else:
                    print("File Exists but Unsupported File Format: ", valid_File)
            else:
                print(f"Error FILES {i+1}: {valid_File}")
                exit()
        print("Valid Files are:\n", "\n".join(valid_Files_List))

    if len(valid_Files_List) > 1:
        if JUST_DOWNLOAD:
            exit()
        if not output_dir:
            output_dir = '/content/drive/MyDrive'
        mergeMKVs(valid_Files_List, output_dir, max_size_gb)

main(MKV_FILES, OUTPUT_DIR, MAX_FileSize)

1 ==> https://brucewayne.tgmkvofcl.workers.dev/225530?hash=AgADFw
2 ==> https://brucewayne.tgmkvofcl.workers.dev/225540?hash=AgADGA
3 ==> https://brucewayne.tgmkvofcl.workers.dev/225537?hash=AgADHg
4 ==> https://brucewayne.tgmkvofcl.workers.dev/225553?hash=AgADJw
5 ==> https://brucewayne.tgmkvofcl.workers.dev/225528?hash=AgADGw
6 ==> https://brucewayne.tgmkvofcl.workers.dev/225547?hash=AgADJA
7 ==> https://brucewayne.tgmkvofcl.workers.dev/225534?hash=AgADHQ
8 ==> https://brucewayne.tgmkvofcl.workers.dev/225526?hash=AgADGg
9 ==> https://brucewayne.tgmkvofcl.workers.dev/225542?hash=AgADIQ
10 ==> https://brucewayne.tgmkvofcl.workers.dev/225545?hash=AgADIg
11 ==> https://brucewayne.tgmkvofcl.workers.dev/225548?hash=AgADJg
12 ==> https://brucewayne.tgmkvofcl.workers.dev/225549?hash=AgADJQ
13 ==> https://brucewayne.tgmkvofcl.workers.dev/225532?hash=AgADHA
14 ==> https://brucewayne.tgmkvofcl.workers.dev/225536?hash=AgADGQ
15 ==> https://brucewayne.tgmkvofcl.workers.dev/225538?hash=AgADHw 





In [ ]:

#@title URL Downloader

URLs = "https://mixologystream-bot-0e080b10c6a4.herokuapp.com/38887/Toonworld4all_I%27m_in_Love_with_the_Villainess_S01E12_720p_.mkv?hash=AgADTx"  # @param {type: "string"}
OUTPUT_DIR = "/content"  #@param {type: "string"}
Select_Downloader = "Aria2os"  #@param ["Aria2", "wget", "Aria2os"]

import subprocess

def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n\n")
    return finalList


def main(urlStr):
    URLs_List = strToList(urlStr, ",")
    for i, _url in enumerate(URLs_List):
        print(f"\n{i+1} ==> {_url}")
        match Select_Downloader:
            case "Aria2":
                !aria2c --continue=true --file-allocation=none -x 16 -s 16 -d "{OUTPUT_DIR}" "{_url}"
            case "wget":
                !wget -d "{OUTPUT_DIR}" "{_url}"
            case "Aria2os":
                cmd = f"aria2c --continue=true --file-allocation=none -x 16 -s 16 -d \"{OUTPUT_DIR}\" \"{_url}\""
                process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                file_path = process.stdout.decode('utf-8')
                file_path = file_path.split("Download complete: ")
                file_path = file_path[1].split("\n")[0]
                print(os.path.exists(file_path))
                print(file_path)


main(URLs)

In [ ]:

#@title Show MKV tracks

FILES_Folder = "/content/Reborn"  #@param {type: "string"}
Output_HTML = True  #@param {type: "boolean"}

from IPython.display import HTML
from pymediainfo import MediaInfo
import os

def get_track_info(file_path):
    try:
        media_info = MediaInfo.parse(file_path)
        tracks_info = []
        for track in media_info.tracks:
            if track.track_type in ["Video", "Audio", "Text"]:  # Only show relevant tracks
                track_id = track.track_id or "Unknown"
                track_type = track.track_type or "Unknown"
                codec = track.codec_id or "Unknown"
                language = track.language or "Unknown"
                title = track.title or "Untitled"
                tracks_info.append((track_id, track_type, codec, language, title))
        return tracks_info
    except Exception as e:
        print(f"Error occurred:\n{e}")
        return []

def getMKVFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort(key=str.lower)
    return all_files

def compare_tracks(tracks1, tracks2):
    return tracks1 == tracks2

def main(folder_path, use_html=True):
    files = getMKVFilesList(folder_path)
    if not files:
        print("No MKV files found in the specified folder.")
        return

    reference_tracks = get_track_info(files[0])
    # HTML output with copy-to-clipboard buttons
    html_output = f"""
    <h3 onclick="copyToClipboard('{files[0]}')">Reference file: <b>{files[0]}</b>
    </h3>
    <ul>
    """
    for track in reference_tracks:
        html_output += f"<li>ID: {track[0]} | Type: {track[1]} | Codec: {track[2]} | Language: {track[3]} | Title: {track[4]}</li>"
        html_output += f"<li><b>Complete INFO: {track}</b></li>"
    html_output += "</ul>"

    for i, _File in enumerate(files[1:], start=1):
        current_tracks = get_track_info(_File)
        if reference_tracks != current_tracks:
            html_output += f"""
            <h3 onclick="copyToClipboard('{_File}')" style='color:red;'>
            {i+1} ==> {_File} (Different Tracks)
            </h3>
            """
        if reference_tracks != current_tracks:
         html_output += "<ul>"
         for track in current_tracks:
             html_output += f"<li>ID: {track[0]} | Type: {track[1]} | Codec: {track[2]} | Language: {track[3]} | Title: {track[4]}</li>"
             html_output += f"<li><b>Complete INFO: {track}</b></li>"
         html_output += "</ul>"

    # Add JavaScript for copying to clipboard
    html_output += """
    <script>
    function copyToClipboard(text) {
        navigator.clipboard.writeText(text).then(function() {
            alert('Copied to clipboard: ' + text);
        }, function(err) {
            alert('Failed to copy: ' + err);
        });
    }
    </script>
    """

    display(HTML(html_output))


main(FILES_Folder, Output_HTML)

In [ ]:


#@title FFmpeg

FILES = "/content/drive/MyDrive/Solo Leveling (2024) S01E01 1080p BluRay 10bit HEVC [Hindi CR AAC 2.0 + English AAC 2.0 + Japanese AAC 2.0] x265 (DarQ-KiNGSMAN)_merged.mkv"  # @param {type: "string"}
FFMPEG_OPTIONS = "-map 0:v -map 0:a: -map 0:s? -c copy"  #@param {type: "string"}
Delete_Extracted_Data = False  #@param {type: "boolean"}
#@markdown ***Start 00:00:00 / End 00:00:10***
trim_start = "01:30:00"  #@param {type: "string"}
trim_end = "01:30:15"  #@param {type: "string"}
JUST_DOWNLOAD= False  #@param {type: "boolean"}
JUST_EXTRACT= False  #@param {type: "boolean"}

from IPython.display import clear_output
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
drive.mount('/content/drive')
clear_output()


def strToList(listStr, separator=","):
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    finalList = list(set(myList)) # Remove duplicates items from list
    showAllElements = "\n".join(f"{i+1} ==> {ele}" for i, ele in enumerate(finalList))
    print(showAllElements, "\n\n")
    return finalList


def printObj(obj):
    objToStr = str(vars(obj))
    print(objToStr)


def getSize(input_file):
    !du -h "{input_file}"
    !du -BM "{input_file}"


def copy(source, destination, isDeleteSource=0):
    if isDeleteSource == 0:
        !rsync -ah --progress "{source}" "{destination}"
    if isDeleteSource == 1:
        !rsync -ah --progress --remove-source-files "{source}" "{destination}"
        print(f"Source deleted: {source}")
    print(f"Transfer Successfull:\nSource => {source}\nDestination => {destination}")


def deleteFile(input_file):
    !rm "{input_file}"
    print(f"File Deleted: {input_file}")


def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")


def printMediaInfo(input_file):
    """Display media information."""
    !mediainfo "{input_file}"


def extract_file_info(file_path):
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def add_suffix_to_file(file_path, suffix="_Suffix"):
    parent_dir_path, file_name = os.path.split(file_path)
    name, ext = os.path.splitext(file_name)
    new_file_name = f"{name}{suffix}{ext}"
    new_file_path = os.path.join(parent_dir_path, new_file_name)
    return new_file_path

def filter_existing_filepath(base_path):
    directory, filename = os.path.split(base_path)
    name, ext = os.path.splitext(filename)

    counter = 1
    new_path = base_path
    while os.path.exists(new_path):
        new_name = f"{name} ({counter}){ext}" if ext else f"{name} ({counter})"
        new_path = os.path.join(directory, new_name)
        counter += 1

    return new_path


def getFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            if relative == 1:
                full_path = os.path.relpath(full_path, folder_path)
            all_files.append(full_path)

    all_files.sort(key=str.lower)
    return all_files


def runCommand(cmd, showLog=0):
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        print(f"Running command: {cmd}")
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()
    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return process.returncode


def filter_Filename(filename):
    MAX_FILENAME_LENGTH = 245
    # Replace unsuitable characters with a hyphen
    invalid_chars = r'[<>:"/\\|?*\']'
    valid_filename = re.sub(invalid_chars, '-', filename)
    # Truncate the filename if it exceeds the maximum length
    if len(valid_filename) > MAX_FILENAME_LENGTH:
        # Preserve the file extension if present
        if '.' in valid_filename:
            name, extension = valid_filename.rsplit('.', 1)
            truncated_name = name[:MAX_FILENAME_LENGTH - len(extension) - 1]  # Reserve space for the dot
            valid_filename = f"{truncated_name}.{extension}"
        else:
            valid_filename = valid_filename[:MAX_FILENAME_LENGTH]

    return valid_filename


def getResponseFileName(url):
    response = requests.get(url, stream=True)

    # Extract the filename from the Content-Disposition header
    filename = response.headers.get('content-disposition', '')
    if 'filename=' in filename:
        filename = filename.split("filename=")[-1].strip('"')
        return filter_Filename(filename)
    return ""


def Aria2Downloader(url, output_dir=''):
    """Download a file using Aria2."""
    print(f"Downloading file: {url}")
    Aria2 = f"aria2c --continue=true --file-allocation=none -x 16 -s 16 -d \"{output_dir}\" \"{url}\""
    process = subprocess.run(Aria2, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if process.returncode == 0:
        print(f"\nDownload Successfull: {output_file}\n")
        output_file_path = process.stdout.decode('utf-8')
        output_file_path = output_file_path.split("Download complete: ")
        output_file_path = output_file_path[1].split("\n")[0]
        print(f"\n\033[92mOutput ==> {output_file_path}\033[0m")  # Green color
        return output_file_path
    else:
        print(f"Download Error Status code({process.returncode}): {url}")

    return url


def useFFmpeg(input_file, output_file='', log=0):
    if not output_file:
        output_file = input_file
    output_file = filter_existing_filepath(output_file)
    global FFMPEG_OPTIONS
    global trim_start
    global trim_end
    trim = ""
    if trim_start:
        trim = f"-ss {trim_start} -to {trim_end}"
    if not FFMPEG_OPTIONS:
        FFMPEG_OPTIONS = "-map 0:v -map 0:a:m:language:hin -map 0:s? -c copy"
    command = f"ffmpeg -i \"{input_file}\" {trim} {FFMPEG_OPTIONS} -y \"{output_file}\""
    runCommand(command, log)
    print("Original Size:")
    getSize(input_file)
    print("NEW Size:")
    getSize(output_file)
    return output_file

def useFFmpegFolder(input_folder, log=0):
    ffmpeg_folder = input_folder + "_ffmpeg"
    if os.path.exists(ffmpeg_folder):
        deleteFolder(ffmpeg_folder)
    os.makedirs(ffmpeg_folder, exist_ok=True)

    for i, file in enumerate(getFilesList(input_folder)):
        if file.endswith(('.mkv', '.mp4', '.avi')):
            output_file = os.path.join(ffmpeg_folder, os.path.split(file)[1])
            print(f"{i+1}: {file}")
            useFFmpeg(file, output_file, log)

    print(f"\nFFmpeg Folder Operation Success: {ffmpeg_folder}\n")
    return ffmpeg_folder


def extractArchive(Archive_File, extracted_folder=''):
    if not extracted_folder:
        extracted_folder = os.path.splitext(Archive_File)[0]
    if os.path.exists(extracted_folder):
        deleteFolder(extracted_folder)
    os.makedirs(extracted_folder, exist_ok=True)
    print(f"\nExtracting archive to: {extracted_folder}")
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    print(f"Extraction complete: {extracted_folder}\n")
    return extracted_folder

def createArchive(_folder, Archive_filepath):
    if os.path.exists(Archive_filepath):
        Archive_filepath = filter_existing_filepath(Archive_filepath)
    print("Building ZIP ===> ", Archive_filepath)
    patoolib.create_archive(Archive_filepath, getFilesList(_folder))
    print(f"Archived Success: {Archive_filepath}")
    return Archive_filepath

def processArchive(Archive_File):
    result_Archive = "/content/drive/MyDrive/" + os.path.split(Archive_File)[1]
    extracted_folder = extractArchive(Archive_File)
    if JUST_EXTRACT:
     return
    ffmpeg_folder = useFFmpegFolder(extracted_folder)
    createArchive(ffmpeg_folder, result_Archive)
    getSize(Archive_File)
    getSize(result_Archive)
    if Delete_Extracted_Data:
        deleteFolder(extracted_folder)
        deleteFolder(ffmpeg_folder)
    return result_Archive


# -------- Main Workflow --------

def main(filesListStr):
    drive_root = '/content/drive/MyDrive/'
    Files_List = strToList(filesListStr)

    for i, _File in enumerate(Files_List):
        if _File.startswith('http'):
            _File = Aria2Downloader(_File)
            if JUST_DOWNLOAD:
                return
        if os.path.exists(_File):
            if _File.endswith('.mkv'):
                output_file = ""
                if not _File.startswith(drive_root):
                    output_file = os.path.join(drive_root, os.path.split(_File)[1])
                useFFmpeg(_File, output_file, 1)
            elif patoolib.is_archive(_File):
                processArchive(_File)
            elif os.path.isdir(_File):
                useFFmpegFolder(_File, 1)
            else:
                print("File Exists but Unsupported File: ", _File)
        else:
            print(f"Error FILES {i+1}: {_File}")

main(FILES)

In [ ]:
#@title File Delete

Delete_File = "/content/Party.Till.I.Die.S01.720p.AMZN.WEB-DL.DDP2.0.H.265-PrimeFix"  #@param {type: "string"}
!ls -lh "{Delete_File}"
!rm -rf "{Delete_File}"

In [ ]:

#@title Save Public Google Drive Files

from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate and initialize
auth.authenticate_user()

# Get credentials
credentials = GoogleCredentials.get_application_default()
service = build('drive', 'v3', credentials=credentials)

public_drive_urls = "https://drive.google.com/file/d/1W04v0CAn8H29VpIIOhs6-3-YVPdLRofm/view"  # @param {type: "string"}
# https://drive.google.com/file/d/13Qdkvg8uKGuZv2_8Z1kyUzCMSSlblEEb/view?usp=drivesdk
# https://drive.google.com/drive/folders/1NrBvFvL-Jro3VQBSQhHiOj4-oEEnCIUl
# https://drive.google.com/drive/folders/1glEPV-ISGauk8_HXQ6dV6AfH1HJRsVc6?usp=sharing


def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    showAllURLs = "\n".join(f"{i+1}. {ele}" for i, ele in enumerate(myList))
    print(showAllURLs)
    return myList


def cleanURL(url):
    if not url:
        return url
    if "." in url:
        if not url.lower().startswith("http"):
            url = "http://" + url

    return url


def extractPublicDriveID(url):
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    try:
        # Check if the URL is a file or folder and extract the ID accordingly
        file_id = ""
        if "file/d/" in url:
            file_id = url.split('/d/')[1].split('/')[0]
        elif "folders/" in url:
            file_id = url.split('folders/')[1].split('/')[0]
        else:
            return ""  # Invalid URL format

        if "?usp=sharing" in file_id:
            file_id = file_id.replace("?usp=sharing","")

        # Create a file object and load metadata using the extracted ID
        file = drive.CreateFile({'id': file_id})
        file.FetchMetadata()  # Fetch metadata to verify ID and access permissions
        return file['id']
    except Exception as e:
        return ""

def getDriveMetadata(_id, metadata_key):
    _metadata = service.files().get(fileId=_id, fields=metadata_key).execute()
    metadata_value = _metadata.get(metadata_key)
    return metadata_value

# Function to create a folder in your Drive
def create_drive_folder(name, parent_id=None):
    folder_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_id] if parent_id else ["root"]
    }
    folder = service.files().create(body=folder_metadata, fields='id').execute()
    return folder.get('id')

# Function to copy a drive file
def transfer_drive_file(file_id, name="", parent_id=None):
    if not name:
        name = getDriveMetadata(file_id, 'name')
    copied_file_metadata = {
        'name': name,
        'parents': [parent_id] if parent_id else ["root"]
    }
    copied_file = service.files().copy(fileId=file_id, body=copied_file_metadata).execute()
    # return name, copied_file.get('id')
    return name

# Function to recursively copy a folder
def transfer_drive_folder(folder_id, parent_id=None):
    # Get the folder metadata
    folder_name = getDriveMetadata(folder_id, 'name')

    # Create the folder in your Drive
    new_folder_id = create_drive_folder(folder_name, parent_id)

    # List all items in the folder
    results = service.files().list(
        q=f"'{folder_id}' in parents",
        fields="files(id, name, mimeType)"
    ).execute()

    items = results.get('files', [])

    for item in items:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            # Recursively copy subfolder
            transfer_drive_folder(item['id'], new_folder_id)
        else:
            # Copy individual file
            transfer_drive_file(item['id'], item['name'], new_folder_id)

    print(f"Copied folder '{folder_name}' to your Drive (ID: {new_folder_id})")


def downloadFromPublicDrive(Drive_URLs):
    colab_root = "/content/"
    drive_root = "/content/drive/MyDrive/"
    file_folder_name = ""
    file_folder_path = ""

    Drive_URLs_List = strToList(Drive_URLs)
    for url in Drive_URLs_List:
        public_drive_id = extractPublicDriveID(url)
        if "file/d/" in url:
            # Start copying the folder
            try:
                file_folder_name = transfer_drive_file(public_drive_id)
                file_folder_path = drive_root + file_folder_name
            except HttpError as error:
                print(f"Drive error occurred: {error}")
        elif "folders/" in url:
            # Start copying the folder
            try:
                transfer_drive_folder(public_drive_id)
                file_folder_name = getDriveMetadata(public_drive_id, 'name')
                file_folder_path = drive_root + file_folder_name
            except HttpError as error:
                print(f"Drive error occurred: {error}")

        print(f"Drive Transfer Successfull: {file_folder_name}")


downloadFromPublicDrive(public_drive_urls)


# _. _. _.  _. _. _.  _. _. _.  _. _. _.  _. _. _.  _. _. _.

In [10]:
!ls -lh "/content/drive/MyDrive/Natsume Yuujinchou S6 - 01 [720p] [Sub] @Anime_Gallery_merged.mkv"

-rw------- 1 root root 739M Jan 29 19:38 '/content/drive/MyDrive/Natsume Yuujinchou S6 - 01 [720p] [Sub] @Anime_Gallery_merged.mkv'
